In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from geopy.exc import GeocoderTimedOut 
from geopy.geocoders import Nominatim

%matplotlib inline

In [2]:
#import NDC deliveries csv
ndc_partner_del = pd.read_csv('../data/ndc_deliveries.csv')
print(ndc_partner_del.shape)
ndc_partner_del.head()

(108, 6)


,Partners,Address,Longitude,Latitude,Total,County
0,10th & Fatherland,"1000 Fatherland St, Nashville, TN 37206",-86.739866,36.173157,3500,"Davidson, TN"
1,Alex Green Elementary,"3921 Lloyd Road, Nashville, TN 37189",-86.835791,36.252676,2250,"Davidson, TN"
2,Antioch High School,"1900 Hobson Pike, Antioch, TN 37013",-86.598459,36.047051,82250,"Davidson, TN"
3,Antioch Middle School,"5050 Blue Hole Rd, Antioch, TN 37013",-86.672124,36.059970,3100,"Davidson, TN"
4,Apollo Middle School,"631 Richards Rd, Nashville, TN 37013",-86.673096,36.068438,5150,"Davidson, TN"


In [3]:
#get total deliveries by county
total_by_county=ndc_partner_del.groupby(['County'], as_index=False)['Total'].sum()
total_by_county

,County,Total
0,"Davidson, TN",1817313
1,"Marshall, TN",2025
2,"Maury, TN",1475
3,"Washington, TN",3750
4,"Williamson, TN",6050
5,"Wilson, TN",10550


In [4]:
#import county file
counties = gpd.read_file('../data/county.geojson')
counties.head()

,GEOID,FIPS_CODE,COUNTY_STATE_CODE,COUNTY_STATE_NAME,geometry
0,01005,01-005,"Barbour, AL","Barbour, Alabama","POLYGON ((-85.65770 31.88030, -85.64880 31.911..."
1,01023,01-023,"Choctaw, AL","Choctaw, Alabama","POLYGON ((-88.47320 31.89390, -88.43130 32.227..."
2,01035,01-035,"Conecuh, AL","Conecuh, Alabama","POLYGON ((-86.90590 31.75300, -86.90650 31.632..."
3,01051,01-051,"Elmore, AL","Elmore, Alabama","POLYGON ((-86.37500 32.75360, -86.00720 32.755..."
4,01065,01-065,"Hale, AL","Hale, Alabama","POLYGON ((-87.71570 33.00680, -87.42190 33.003..."


In [5]:
# keep only rows for counties needed
counties=counties[counties['COUNTY_STATE_CODE'].isin(['Davidson, TN','Wilson, TN','Williamson, TN','Washington, TN','Marshall, TN','Maury, TN'])]
counties.head()

,GEOID,FIPS_CODE,COUNTY_STATE_CODE,COUNTY_STATE_NAME,geometry
661,47037,47-037,"Davidson, TN","Davidson, Tennessee","POLYGON ((-86.91320 36.38260, -86.85430 36.369..."
673,47189,47-189,"Wilson, TN","Wilson, Tennessee","POLYGON ((-86.59430 36.24510, -86.55980 36.287..."
870,47117,47-117,"Marshall, TN","Marshall, Tennessee","POLYGON ((-86.96080 35.41700, -86.95500 35.451..."
1023,47187,47-187,"Williamson, TN","Williamson, Tennessee","POLYGON ((-87.20420 35.95920, -87.18260 36.049..."
2257,47179,47-179,"Washington, TN","Washington, Tennessee","POLYGON ((-82.68070 36.43180, -82.65800 36.438..."


In [6]:
#create TN counties
polygon_tn = counties
polygon_tn.shape

(6, 5)

In [8]:
ndc_map = folium.Map(
    location=[35.85,-86.767960],
    zoom_start=8.5,
    #tiles='Stamen Terrain'
)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(ndc_map)

folium.GeoJson(polygon_tn).add_to(ndc_map)

folium.Choropleth(
    geo_data=counties,
    name='choropleth',
    data=total_by_county,
    columns=['County', 'Total'],
    key_on='feature.properties.COUNTY_STATE_CODE',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Diapers Distributed to Partners 2020 YTD'
).add_to(ndc_map)

folium.LayerControl().add_to(ndc_map)

# inside the loop add each marker to the cluster
for row_index, row_values in ndc_partner_del.iterrows():
    loc = [row_values['Latitude'], row_values['Longitude']]
    pop = '<b>Partner:</b> ' + str(row_values['Partners']) + ' ' + '<b>Diapers Delivered:</b> ' + str(row_values['Total'])
    icon=folium.Icon(color="orange",icon="circle", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)
    
ndc_map